In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import StackingRegressor, VotingRegressor

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [2]:
df2017 = pd.read_csv('./test2017.csv', sep=',', index_col=0)
df2018 = pd.read_csv('./test2018.csv', sep=',', index_col=0)
df2019 = pd.read_csv('./test2019.csv', sep=',', index_col=0)
df2020 = pd.read_csv('./test2020.csv', sep=',', index_col=0)
df2021 = pd.read_csv('./test2021.csv', sep=',', index_col=0)

In [3]:
df_train = pd.concat([df2017, df2018, df2019, df2020, df2021])

In [4]:
df_train = df_train.reset_index()
df_train['기준일ID'] = df_train['기준일ID'].astype('str')
df_train['기준일ID'] = pd.to_datetime(df_train['기준일ID'])
df_train['year'] = df_train['기준일ID'].dt.year
df_train['month'] = df_train['기준일ID'].dt.month
df_train['day'] = df_train['기준일ID'].dt.day

df_train = df_train.loc[:,['총생활인구수', '시간대구분', 'year', 'month', 'day']]
df_train.rename(columns={'시간대구분':'hour'}, inplace=True)

In [5]:
df_train

,총생활인구수,hour,year,month,day
0,43922.2281,0,2017,1,1
1,43763.4929,1,2017,1,1
2,43561.5708,2,2017,1,1
3,43928.9929,3,2017,1,1
4,44150.2973,4,2017,1,1
...,...,...,...,...,...
43507,33304.5400,19,2021,12,31
43508,33368.7201,20,2021,12,31
43509,34083.1721,21,2021,12,31
43510,34380.0806,22,2021,12,31


In [6]:
month = [1, 2]
day = range(1,32)
year = 2022
hour = range(24)
ls = []
for m in month:
  for d in day:
    for h in hour:
      ls.append((year,m,d,h))

ls = ls[:-72]
len(ls)

1416

In [7]:
temp = []
for year, m, d, h in ls:
  temp.append([np.nan, year, m, d, h])
df_test = pd.DataFrame(temp, columns=list(df_train))
df_test.rename(columns={'hour':'year','year':'month','month':'day','day':'hour'},inplace=True)

In [8]:
df_total_new = pd.concat([df_train, df_test])
df_total_new.reset_index(drop=True, inplace=True)
df_total_new

,총생활인구수,hour,year,month,day
0,43922.2281,0,2017,1,1
1,43763.4929,1,2017,1,1
2,43561.5708,2,2017,1,1
3,43928.9929,3,2017,1,1
4,44150.2973,4,2017,1,1
...,...,...,...,...,...
44923,NaN,19,2022,2,28
44924,NaN,20,2022,2,28
44925,NaN,21,2022,2,28
44926,NaN,22,2022,2,28


In [9]:
from datetime import date
def is_weekend(d):
    return d.weekday() > 4
weekend_ls = []
date_ls = []
for y,m,d in zip(df_total_new['year'], df_total_new['month'], df_total_new['day']):
  weekend_ls.append(int(is_weekend(date(y,m,d))))
  date_ls.append(str(date(y,m,d)))
df_total_new['weekend'] = weekend_ls
df_total_new['날짜'] = date_ls

In [10]:
df_total_new['날짜'] = pd.to_datetime(df_total_new['날짜'], format = '%Y-%m-%d')

In [11]:
df_total_new

,총생활인구수,hour,year,month,day,weekend,날짜
0,43922.2281,0,2017,1,1,1,2017-01-01
1,43763.4929,1,2017,1,1,1,2017-01-01
2,43561.5708,2,2017,1,1,1,2017-01-01
3,43928.9929,3,2017,1,1,1,2017-01-01
4,44150.2973,4,2017,1,1,1,2017-01-01
...,...,...,...,...,...,...,...
44923,NaN,19,2022,2,28,0,2022-02-28
44924,NaN,20,2022,2,28,0,2022-02-28
44925,NaN,21,2022,2,28,0,2022-02-28
44926,NaN,22,2022,2,28,0,2022-02-28


In [12]:
df_total_new['공휴일'] = False
df_total_new['chusuk_lunar'] = False
df_total_new['lunar'] = False

In [13]:
df_total_new

,총생활인구수,hour,year,month,day,weekend,날짜,공휴일,chusuk_lunar,lunar
0,43922.2281,0,2017,1,1,1,2017-01-01,False,False,False
1,43763.4929,1,2017,1,1,1,2017-01-01,False,False,False
2,43561.5708,2,2017,1,1,1,2017-01-01,False,False,False
3,43928.9929,3,2017,1,1,1,2017-01-01,False,False,False
4,44150.2973,4,2017,1,1,1,2017-01-01,False,False,False
...,...,...,...,...,...,...,...,...,...,...
44923,NaN,19,2022,2,28,0,2022-02-28,False,False,False
44924,NaN,20,2022,2,28,0,2022-02-28,False,False,False
44925,NaN,21,2022,2,28,0,2022-02-28,False,False,False
44926,NaN,22,2022,2,28,0,2022-02-28,False,False,False


In [14]:
df_total_new.loc[df_total_new['날짜']=='2017-01-27','lunar']=True
df_total_new.loc[df_total_new['날짜']=='2017-01-28','lunar']=True
df_total_new.loc[df_total_new['날짜']=='2017-01-29','lunar']=True
df_total_new.loc[df_total_new['날짜']=='2018-02-15','lunar']=True
df_total_new.loc[df_total_new['날짜']=='2018-02-16','lunar']=True
df_total_new.loc[df_total_new['날짜']=='2018-02-17','lunar']=True
df_total_new.loc[df_total_new['날짜']=='2019-02-04','lunar']=True
df_total_new.loc[df_total_new['날짜']=='2019-02-05','lunar']=True
df_total_new.loc[df_total_new['날짜']=='2019-02-06','lunar']=True
df_total_new.loc[df_total_new['날짜']=='2020-01-24','lunar']=True
df_total_new.loc[df_total_new['날짜']=='2020-01-25','lunar']=True
df_total_new.loc[df_total_new['날짜']=='2020-01-26','lunar']=True
df_total_new.loc[df_total_new['날짜']=='2021-01-31','lunar']=True
df_total_new.loc[df_total_new['날짜']=='2021-02-01','lunar']=True
df_total_new.loc[df_total_new['날짜']=='2021-02-02','lunar']=True
df_total_new.loc[df_total_new['날짜']=='2022-01-31','lunar']=True
df_total_new.loc[df_total_new['날짜']=='2022-02-01','lunar']=True
df_total_new.loc[df_total_new['날짜']=='2022-02-02','lunar']=True


df_total_new.loc[df_total_new['날짜']=='2017-01-27','chusuk_lunar']=True
df_total_new.loc[df_total_new['날짜']=='2017-01-28','chusuk_lunar']=True
df_total_new.loc[df_total_new['날짜']=='2017-01-29','chusuk_lunar']=True
df_total_new.loc[df_total_new['날짜']=='2018-02-15','chusuk_lunar']=True
df_total_new.loc[df_total_new['날짜']=='2018-02-16','chusuk_lunar']=True
df_total_new.loc[df_total_new['날짜']=='2018-02-17','chusuk_lunar']=True
df_total_new.loc[df_total_new['날짜']=='2019-02-04','chusuk_lunar']=True
df_total_new.loc[df_total_new['날짜']=='2019-02-05','chusuk_lunar']=True
df_total_new.loc[df_total_new['날짜']=='2019-02-06','chusuk_lunar']=True
df_total_new.loc[df_total_new['날짜']=='2020-01-24','chusuk_lunar']=True
df_total_new.loc[df_total_new['날짜']=='2020-01-25','chusuk_lunar']=True
df_total_new.loc[df_total_new['날짜']=='2020-01-26','chusuk_lunar']=True
df_total_new.loc[df_total_new['날짜']=='2021-01-31','chusuk_lunar']=True
df_total_new.loc[df_total_new['날짜']=='2021-02-01','chusuk_lunar']=True
df_total_new.loc[df_total_new['날짜']=='2021-02-02','chusuk_lunar']=True
df_total_new.loc[df_total_new['날짜']=='2022-01-31','chusuk_lunar']=True
df_total_new.loc[df_total_new['날짜']=='2022-02-01','chusuk_lunar']=True
df_total_new.loc[df_total_new['날짜']=='2022-02-02','chusuk_lunar']=True

df_total_new.loc[df_total_new['날짜']=='2017-10-03','chusuk_lunar']=True
df_total_new.loc[df_total_new['날짜']=='2017-10-04','chusuk_lunar']=True
df_total_new.loc[df_total_new['날짜']=='2017-10-05','chusuk_lunar']=True
df_total_new.loc[df_total_new['날짜']=='2018-09-23','chusuk_lunar']=True
df_total_new.loc[df_total_new['날짜']=='2018-09-24','chusuk_lunar']=True
df_total_new.loc[df_total_new['날짜']=='2018-09-25','chusuk_lunar']=True
df_total_new.loc[df_total_new['날짜']=='2019-09-12','chusuk_lunar']=True
df_total_new.loc[df_total_new['날짜']=='2019-09-13','chusuk_lunar']=True
df_total_new.loc[df_total_new['날짜']=='2019-09-14','chusuk_lunar']=True
df_total_new.loc[df_total_new['날짜']=='2020-09-30','chusuk_lunar']=True
df_total_new.loc[df_total_new['날짜']=='2020-10-01','chusuk_lunar']=True
df_total_new.loc[df_total_new['날짜']=='2020-10-02','chusuk_lunar']=True
df_total_new.loc[df_total_new['날짜']=='2020-10-03','chusuk_lunar']=True
df_total_new.loc[df_total_new['날짜']=='2021-09-20','chusuk_lunar']=True
df_total_new.loc[df_total_new['날짜']=='2021-09-21','chusuk_lunar']=True
df_total_new.loc[df_total_new['날짜']=='2021-09-22','chusuk_lunar']=True

df_total_new.loc[df_total_new['날짜']=='2017-01-01','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2017-01-27','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2017-01-28','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2017-01-29','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2017-03-01','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2017-05-03','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2017-05-05','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2017-05-09','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2017-06-06','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2017-08-15','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2017-10-03','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2017-10-04','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2017-10-05','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2017-10-09','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2017-12-25','공휴일']=True

df_total_new.loc[df_total_new['날짜']=='2018-01-01','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2018-02-15','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2018-02-16','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2018-02-17','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2018-03-01','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2018-05-05','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2018-05-22','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2018-06-06','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2018-08-15','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2018-09-23','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2018-09-24','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2018-09-25','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2018-10-03','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2018-10-09','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2018-12-25','공휴일']=True

df_total_new.loc[df_total_new['날짜']=='2019-01-01','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2019-02-04','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2019-02-05','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2019-02-06','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2019-03-01','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2019-05-05','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2019-05-12','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2019-06-06','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2019-08-15','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2019-09-12','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2019-09-13','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2019-09-14','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2019-10-03','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2019-10-09','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2019-12-25','공휴일']=True

df_total_new.loc[df_total_new['날짜']=='2020-01-01','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2020-01-24','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2020-01-25','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2020-01-26','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2020-03-01','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2020-04-15','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2020-04-30','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2020-05-05','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2020-06-06','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2020-08-15','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2020-09-30','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2020-10-01','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2020-10-02','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2020-10-03','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2020-10-09','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2020-12-25','공휴일']=True

df_total_new.loc[df_total_new['날짜']=='2021-01-01','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2021-02-11','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2021-02-12','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2021-02-13','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2021-03-01','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2021-05-05','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2021-05-19','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2021-06-06','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2021-08-15','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2021-09-20','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2021-09-21','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2021-09-22','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2021-10-03','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2021-10-09','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2021-12-25','공휴일']=True

df_total_new.loc[df_total_new['날짜']=='2022-01-01','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2022-01-31','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2022-02-01','공휴일']=True
df_total_new.loc[df_total_new['날짜']=='2022-02-02','공휴일']=True

In [16]:
import requests
import xml.etree.ElementTree as ET
 
year=[2017,2018,2019,2020,2021]
key = "tYQoDIlfuqQdGatJQ8Bonl2C4PzpWEeiMsQ8o4QxuRGwdg%2BNneMjn6H6wwRosTgV4pRf5p39OxZrPYya0XWjew%3D%3D"
holidays = []
for yr in year:
  res=requests.get("http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getHoliDeInfo?solYear="+str(yr)+"&numOfRows=100&ServiceKey="+key) #한국천문연구원_특일 정보
  res.raise_for_status()
  res.encoding='EUC.KR'
  res=res.text

  temp=[]

  root = ET.fromstring(res) 
  for data in root.iter('item'): 
      
      isholiday=data[2].text
      if isholiday=="Y":
          temp.append(data[3].text)
  holidays.append(temp)

holi = []
for ls in holidays:
  for day in ls:
    holi.append(day)

is_holiday = []
for y, m, d in zip(df_total_new['year'], df_total_new['month'], df_total_new['day']):
  date = str(y) + "0"*(2-len(str(m))) + str(m) + "0"*(2-len(str(d))) + str(d)
  if date in holi:
    is_holiday.append(1)
  else:
    is_holiday.append(0)

df_total_new['holiday'] = is_holiday

In [17]:
df_total_new['공휴일']=df_total_new['공휴일'].astype('int')
df_total_new['lunar']=df_total_new['lunar'].astype('int')
df_total_new['chusuk_lunar']=df_total_new['chusuk_lunar'].astype('int')

In [19]:
df_total_new['shift1'] = df_total_new['총생활인구수'].shift(1416)
df_total_new['shift2'] = df_total_new['총생활인구수'].shift(1417)
df_total_new['shift3'] = df_total_new['총생활인구수'].shift(1418)
df_total_new['shift4'] = df_total_new['총생활인구수'].shift(1419)
df_total_new['shift5'] = df_total_new['총생활인구수'].shift(1420)
df_total_new['shift6'] = df_total_new['총생활인구수'].shift(1421)
df_total_new['shift7'] = df_total_new['총생활인구수'].shift(1422)
df_total_new['shift8'] = df_total_new['총생활인구수'].shift(1423)
df_total_new['shift9'] = df_total_new['총생활인구수'].shift(1424)
df_total_new['shift10'] = df_total_new['총생활인구수'].shift(1425)
df_total_new['shift11'] = df_total_new['총생활인구수'].shift(1426)
df_total_new['shift12'] = df_total_new['총생활인구수'].shift(1427)
df_total_new['shift13'] = df_total_new['총생활인구수'].shift(1428)
df_total_new['shift14'] = df_total_new['총생활인구수'].shift(1429)
df_total_new['shift15'] = df_total_new['총생활인구수'].shift(1430)
df_total_new['shift16'] = df_total_new['총생활인구수'].shift(1431)
df_total_new['shift17'] = df_total_new['총생활인구수'].shift(1432)
df_total_new['shift18'] = df_total_new['총생활인구수'].shift(1433)
df_total_new['shift19'] = df_total_new['총생활인구수'].shift(1434)
df_total_new['shift20'] = df_total_new['총생활인구수'].shift(1435)
df_total_new['shift21'] = df_total_new['총생활인구수'].shift(1436)
df_total_new['shift22'] = df_total_new['총생활인구수'].shift(1437)
df_total_new['shift23'] = df_total_new['총생활인구수'].shift(1438)
df_total_new['shift24'] = df_total_new['총생활인구수'].shift(1439)
df_total_new['shift25'] = df_total_new['총생활인구수'].shift(1440)
df_total_new['shift26'] = df_total_new['총생활인구수'].shift(1441)
df_total_new['shift27'] = df_total_new['총생활인구수'].shift(1442)
df_total_new['shift28'] = df_total_new['총생활인구수'].shift(1443)
df_total_new['shift29'] = df_total_new['총생활인구수'].shift(1444)
df_total_new['shift30'] = df_total_new['총생활인구수'].shift(1445)

In [20]:
df_total_new['이동평균1'] = df_total_new['총생활인구수'].shift(1416).rolling(1416).mean()
df_total_new['이동평균2'] = df_total_new['총생활인구수'].shift(1417).rolling(1417).mean()
df_total_new['이동평균3'] = df_total_new['총생활인구수'].shift(1418).rolling(1418).mean()
df_total_new['이동평균4'] = df_total_new['총생활인구수'].shift(1419).rolling(1419).mean()
df_total_new['이동평균5'] = df_total_new['총생활인구수'].shift(1420).rolling(1420).mean()
df_total_new['이동평균6'] = df_total_new['총생활인구수'].shift(1421).rolling(1421).mean()
df_total_new['이동평균7'] = df_total_new['총생활인구수'].shift(1422).rolling(1422).mean()
df_total_new['이동평균8'] = df_total_new['총생활인구수'].shift(1423).rolling(1423).mean()
df_total_new['이동평균9'] = df_total_new['총생활인구수'].shift(1424).rolling(1424).mean()
df_total_new['이동평균10'] = df_total_new['총생활인구수'].shift(1425).rolling(1425).mean()
df_total_new['이동평균11'] = df_total_new['총생활인구수'].shift(1426).rolling(1426).mean()
df_total_new['이동평균12'] = df_total_new['총생활인구수'].shift(1427).rolling(1427).mean()
df_total_new['이동평균13'] = df_total_new['총생활인구수'].shift(1428).rolling(1428).mean()
df_total_new['이동평균14'] = df_total_new['총생활인구수'].shift(1429).rolling(1429).mean()
df_total_new['이동평균15'] = df_total_new['총생활인구수'].shift(1430).rolling(1430).mean()
df_total_new['이동평균16'] = df_total_new['총생활인구수'].shift(1431).rolling(1431).mean()
df_total_new['이동평균17'] = df_total_new['총생활인구수'].shift(1432).rolling(1432).mean()
df_total_new['이동평균18'] = df_total_new['총생활인구수'].shift(1433).rolling(1433).mean()
df_total_new['이동평균19'] = df_total_new['총생활인구수'].shift(1434).rolling(1434).mean()
df_total_new['이동평균20'] = df_total_new['총생활인구수'].shift(1435).rolling(1435).mean()
df_total_new['이동평균21'] = df_total_new['총생활인구수'].shift(1436).rolling(1436).mean()
df_total_new['이동평균22'] = df_total_new['총생활인구수'].shift(1437).rolling(1437).mean()
df_total_new['이동평균23'] = df_total_new['총생활인구수'].shift(1438).rolling(1438).mean()
df_total_new['이동평균24'] = df_total_new['총생활인구수'].shift(1439).rolling(1439).mean()
df_total_new['이동평균25'] = df_total_new['총생활인구수'].shift(1440).rolling(1440).mean()
df_total_new['이동평균26'] = df_total_new['총생활인구수'].shift(1441).rolling(1441).mean()
df_total_new['이동평균27'] = df_total_new['총생활인구수'].shift(1442).rolling(1442).mean()
df_total_new['이동평균28'] = df_total_new['총생활인구수'].shift(1443).rolling(1443).mean()
df_total_new['이동평균29'] = df_total_new['총생활인구수'].shift(1444).rolling(1444).mean()
df_total_new['이동평균30'] = df_total_new['총생활인구수'].shift(1445).rolling(1445).mean()

In [21]:
df_total_new.drop(columns=['year','month', 'day', '날짜'], inplace=True)

In [23]:
train = df_total_new.iloc[:-1416]
test = df_total_new.iloc[-1416:]

train.dropna(inplace=True)
train.reset_index(drop=True, inplace=True)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_6124\1829793219.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.dropna(inplace=True)


In [24]:
x_train = train.drop(columns=['총생활인구수'])
y_train = train.loc[:, '총생활인구수']

test.reset_index(inplace=True, drop=True)
y_test = test['총생활인구수']
x_test = test.drop(columns=['총생활인구수'])

In [25]:
models = {
    #여러가지 모델 사용해 보면서 가장 성능이 좋은 것 선정
    #'xgb': XGBRegressor(),
    #'Knb': KNeighborsRegressor,
    'cat': CatBoostRegressor(),
    'rf': RandomForestRegressor(),
    'lgbm': LGBMRegressor(),
}

stacking = StackingRegressor(
    
    
    
    estimators=list(models.items()),
    final_estimator=LinearRegression(),
    cv=5
)
stacking.fit(x_train, y_train)
pred_stacking = stacking.predict(x_test)

Learning rate set to 0.073513
0:	learn: 3526.3811142	total: 212ms	remaining: 3m 31s
1:	learn: 3366.6440307	total: 273ms	remaining: 2m 16s
2:	learn: 3223.2712865	total: 331ms	remaining: 1m 50s
3:	learn: 3093.9389811	total: 358ms	remaining: 1m 29s
4:	learn: 2973.8272569	total: 383ms	remaining: 1m 16s
5:	learn: 2864.0970960	total: 411ms	remaining: 1m 8s
6:	learn: 2768.4581067	total: 439ms	remaining: 1m 2s
7:	learn: 2682.3475035	total: 462ms	remaining: 57.3s
8:	learn: 2601.1848362	total: 487ms	remaining: 53.6s
9:	learn: 2528.8192398	total: 510ms	remaining: 50.5s
10:	learn: 2462.7757191	total: 537ms	remaining: 48.3s
11:	learn: 2402.9947531	total: 563ms	remaining: 46.3s
12:	learn: 2346.1148302	total: 589ms	remaining: 44.7s
13:	learn: 2297.3551902	total: 616ms	remaining: 43.4s
14:	learn: 2255.5117501	total: 641ms	remaining: 42.1s
15:	learn: 2216.0997812	total: 667ms	remaining: 41s
16:	learn: 2179.9630723	total: 692ms	remaining: 40s
17:	learn: 2146.3568016	total: 718ms	remaining: 39.2s
18:	lea

153:	learn: 1578.3856390	total: 4.07s	remaining: 22.4s
154:	learn: 1577.5299726	total: 4.09s	remaining: 22.3s
155:	learn: 1574.9300943	total: 4.12s	remaining: 22.3s
156:	learn: 1574.4949577	total: 4.14s	remaining: 22.2s
157:	learn: 1574.0432813	total: 4.17s	remaining: 22.2s
158:	learn: 1571.8965257	total: 4.19s	remaining: 22.2s
159:	learn: 1569.6090712	total: 4.22s	remaining: 22.1s
160:	learn: 1568.5282250	total: 4.24s	remaining: 22.1s
161:	learn: 1567.3853037	total: 4.27s	remaining: 22.1s
162:	learn: 1565.0567278	total: 4.29s	remaining: 22s
163:	learn: 1563.3527303	total: 4.32s	remaining: 22s
164:	learn: 1561.9754413	total: 4.34s	remaining: 22s
165:	learn: 1561.5248698	total: 4.36s	remaining: 21.9s
166:	learn: 1558.7986000	total: 4.39s	remaining: 21.9s
167:	learn: 1554.6830416	total: 4.41s	remaining: 21.8s
168:	learn: 1553.5132566	total: 4.43s	remaining: 21.8s
169:	learn: 1551.5772123	total: 4.46s	remaining: 21.8s
170:	learn: 1550.4445397	total: 4.48s	remaining: 21.7s
171:	learn: 1549

303:	learn: 1410.7555311	total: 7.71s	remaining: 17.7s
304:	learn: 1410.3735518	total: 7.73s	remaining: 17.6s
305:	learn: 1409.1806494	total: 7.76s	remaining: 17.6s
306:	learn: 1408.0473901	total: 7.78s	remaining: 17.6s
307:	learn: 1406.9538365	total: 7.81s	remaining: 17.5s
308:	learn: 1406.3639494	total: 7.83s	remaining: 17.5s
309:	learn: 1405.5139525	total: 7.85s	remaining: 17.5s
310:	learn: 1404.8509765	total: 7.88s	remaining: 17.5s
311:	learn: 1403.8358169	total: 7.91s	remaining: 17.4s
312:	learn: 1403.3788039	total: 7.93s	remaining: 17.4s
313:	learn: 1401.4148372	total: 7.95s	remaining: 17.4s
314:	learn: 1400.7508731	total: 7.98s	remaining: 17.4s
315:	learn: 1399.3923534	total: 8.01s	remaining: 17.3s
316:	learn: 1398.6052871	total: 8.03s	remaining: 17.3s
317:	learn: 1397.4146672	total: 8.06s	remaining: 17.3s
318:	learn: 1396.0302556	total: 8.09s	remaining: 17.3s
319:	learn: 1395.3560863	total: 8.11s	remaining: 17.2s
320:	learn: 1394.9884135	total: 8.13s	remaining: 17.2s
321:	learn

458:	learn: 1290.9461848	total: 11.6s	remaining: 13.6s
459:	learn: 1290.3707232	total: 11.6s	remaining: 13.6s
460:	learn: 1289.5503300	total: 11.6s	remaining: 13.6s
461:	learn: 1289.1654274	total: 11.6s	remaining: 13.5s
462:	learn: 1288.7395312	total: 11.7s	remaining: 13.5s
463:	learn: 1288.2305712	total: 11.7s	remaining: 13.5s
464:	learn: 1287.6379897	total: 11.7s	remaining: 13.5s
465:	learn: 1287.1450434	total: 11.7s	remaining: 13.4s
466:	learn: 1286.5093859	total: 11.8s	remaining: 13.4s
467:	learn: 1285.6500923	total: 11.8s	remaining: 13.4s
468:	learn: 1284.7674225	total: 11.8s	remaining: 13.4s
469:	learn: 1284.2863544	total: 11.8s	remaining: 13.3s
470:	learn: 1283.6690534	total: 11.9s	remaining: 13.3s
471:	learn: 1283.0523547	total: 11.9s	remaining: 13.3s
472:	learn: 1282.5498745	total: 11.9s	remaining: 13.3s
473:	learn: 1282.0977921	total: 11.9s	remaining: 13.3s
474:	learn: 1281.8270957	total: 12s	remaining: 13.2s
475:	learn: 1281.4501966	total: 12s	remaining: 13.2s
476:	learn: 12

610:	learn: 1206.6224646	total: 15.3s	remaining: 9.71s
611:	learn: 1206.1828230	total: 15.3s	remaining: 9.69s
612:	learn: 1205.5903920	total: 15.3s	remaining: 9.66s
613:	learn: 1205.2459910	total: 15.3s	remaining: 9.64s
614:	learn: 1204.5848290	total: 15.4s	remaining: 9.62s
615:	learn: 1204.2971000	total: 15.4s	remaining: 9.59s
616:	learn: 1203.6097823	total: 15.4s	remaining: 9.57s
617:	learn: 1203.1958856	total: 15.4s	remaining: 9.55s
618:	learn: 1202.5804718	total: 15.5s	remaining: 9.53s
619:	learn: 1202.0683216	total: 15.5s	remaining: 9.5s
620:	learn: 1200.5951063	total: 15.5s	remaining: 9.48s
621:	learn: 1200.1299411	total: 15.6s	remaining: 9.45s
622:	learn: 1199.5039037	total: 15.6s	remaining: 9.43s
623:	learn: 1199.1003788	total: 15.6s	remaining: 9.4s
624:	learn: 1198.7577834	total: 15.6s	remaining: 9.38s
625:	learn: 1198.1906163	total: 15.7s	remaining: 9.35s
626:	learn: 1197.7266442	total: 15.7s	remaining: 9.33s
627:	learn: 1197.2654855	total: 15.7s	remaining: 9.3s
628:	learn: 1

...

268:	learn: 1512.8553798	total: 5.96s	remaining: 16.2s
269:	learn: 1511.8959150	total: 5.98s	remaining: 16.2s
270:	learn: 1510.8631256	total: 6s	remaining: 16.1s
271:	learn: 1509.6358901	total: 6.02s	remaining: 16.1s
272:	learn: 1508.4109569	total: 6.04s	remaining: 16.1s
273:	learn: 1507.3155083	total: 6.06s	remaining: 16.1s
274:	learn: 1505.7491560	total: 6.08s	remaining: 16s
275:	learn: 1504.6109831	total: 6.1s	remaining: 16s
276:	learn: 1503.5090888	total: 6.13s	remaining: 16s
277:	learn: 1502.2267618	total: 6.15s	remaining: 16s
278:	learn: 1501.1126825	total: 6.17s	remaining: 15.9s
279:	learn: 1500.4827742	total: 6.19s	remaining: 15.9s
280:	learn: 1499.8861498	total: 6.21s	remaining: 15.9s
281:	learn: 1499.1887425	total: 6.23s	remaining: 15.9s
282:	learn: 1498.0180079	total: 6.25s	remaining: 15.8s
283:	learn: 1496.4183417	total: 6.28s	remaining: 15.8s
284:	learn: 1495.4825195	total: 6.29s	remaining: 15.8s
285:	learn: 1495.1005632	total: 6.31s	remaining: 15.8s
286:	learn: 1493.70762

418:	learn: 1382.2524150	total: 9.13s	remaining: 12.7s
419:	learn: 1381.4391953	total: 9.17s	remaining: 12.7s
420:	learn: 1380.6496754	total: 9.21s	remaining: 12.7s
421:	learn: 1380.2447220	total: 9.24s	remaining: 12.7s
422:	learn: 1379.5779278	total: 9.27s	remaining: 12.6s
423:	learn: 1378.8394037	total: 9.31s	remaining: 12.7s
424:	learn: 1378.2907706	total: 9.38s	remaining: 12.7s
425:	learn: 1377.9452891	total: 9.41s	remaining: 12.7s
426:	learn: 1377.0567245	total: 9.45s	remaining: 12.7s
427:	learn: 1376.4296783	total: 9.47s	remaining: 12.7s
428:	learn: 1375.8604661	total: 9.51s	remaining: 12.7s
429:	learn: 1375.0433220	total: 9.54s	remaining: 12.6s
430:	learn: 1374.0980038	total: 9.56s	remaining: 12.6s
431:	learn: 1373.5529932	total: 9.59s	remaining: 12.6s
432:	learn: 1372.8197692	total: 9.61s	remaining: 12.6s
433:	learn: 1372.2880424	total: 9.63s	remaining: 12.6s
434:	learn: 1371.3924109	total: 9.65s	remaining: 12.5s
435:	learn: 1370.5257874	total: 9.67s	remaining: 12.5s
436:	learn

568:	learn: 1286.7187951	total: 12.6s	remaining: 9.52s
569:	learn: 1286.4561207	total: 12.6s	remaining: 9.5s
570:	learn: 1285.9102630	total: 12.6s	remaining: 9.48s
571:	learn: 1285.1542545	total: 12.7s	remaining: 9.47s
572:	learn: 1284.2804304	total: 12.7s	remaining: 9.45s
573:	learn: 1283.6678499	total: 12.7s	remaining: 9.43s
574:	learn: 1283.0136516	total: 12.7s	remaining: 9.42s
575:	learn: 1282.5026376	total: 12.8s	remaining: 9.4s
576:	learn: 1282.1810064	total: 12.8s	remaining: 9.39s
577:	learn: 1281.5600856	total: 12.8s	remaining: 9.37s
578:	learn: 1280.6655165	total: 12.9s	remaining: 9.35s
579:	learn: 1279.6493359	total: 12.9s	remaining: 9.34s
580:	learn: 1278.8299407	total: 12.9s	remaining: 9.32s
581:	learn: 1278.0094568	total: 12.9s	remaining: 9.3s
582:	learn: 1277.4812480	total: 13s	remaining: 9.28s
583:	learn: 1276.6492520	total: 13s	remaining: 9.27s
584:	learn: 1276.1472240	total: 13s	remaining: 9.25s
585:	learn: 1275.8228473	total: 13.1s	remaining: 9.23s
586:	learn: 1275.35

725:	learn: 1200.9326229	total: 16.3s	remaining: 6.13s
726:	learn: 1200.4481344	total: 16.3s	remaining: 6.11s
727:	learn: 1199.9601539	total: 16.3s	remaining: 6.09s
728:	learn: 1199.4738431	total: 16.3s	remaining: 6.07s
729:	learn: 1199.1791154	total: 16.4s	remaining: 6.05s
730:	learn: 1198.8330585	total: 16.4s	remaining: 6.03s
731:	learn: 1198.2549077	total: 16.4s	remaining: 6s
732:	learn: 1197.7012977	total: 16.4s	remaining: 5.98s
733:	learn: 1197.4625922	total: 16.4s	remaining: 5.96s
734:	learn: 1197.2501138	total: 16.5s	remaining: 5.93s
735:	learn: 1196.6097153	total: 16.5s	remaining: 5.91s
736:	learn: 1196.4279970	total: 16.5s	remaining: 5.89s
737:	learn: 1195.5712816	total: 16.5s	remaining: 5.87s
738:	learn: 1195.2355629	total: 16.6s	remaining: 5.85s
739:	learn: 1194.9987551	total: 16.6s	remaining: 5.83s
740:	learn: 1194.5185831	total: 16.6s	remaining: 5.81s
741:	learn: 1194.3362671	total: 16.7s	remaining: 5.79s
742:	learn: 1193.9248622	total: 16.7s	remaining: 5.77s
743:	learn: 1

879:	learn: 1133.5647445	total: 19.9s	remaining: 2.72s
880:	learn: 1132.8298646	total: 19.9s	remaining: 2.69s
881:	learn: 1132.4477486	total: 20s	remaining: 2.67s
882:	learn: 1132.2163761	total: 20s	remaining: 2.65s
883:	learn: 1131.5345426	total: 20s	remaining: 2.62s
884:	learn: 1131.1192205	total: 20s	remaining: 2.6s
885:	learn: 1130.6931840	total: 20s	remaining: 2.58s
886:	learn: 1130.2598226	total: 20.1s	remaining: 2.56s
887:	learn: 1129.5720503	total: 20.1s	remaining: 2.53s
888:	learn: 1129.3650529	total: 20.1s	remaining: 2.51s
889:	learn: 1129.1041448	total: 20.1s	remaining: 2.49s
890:	learn: 1128.5756535	total: 20.1s	remaining: 2.46s
891:	learn: 1128.3784230	total: 20.2s	remaining: 2.44s
892:	learn: 1128.0994895	total: 20.2s	remaining: 2.42s
893:	learn: 1127.9302584	total: 20.2s	remaining: 2.4s
894:	learn: 1127.5368949	total: 20.2s	remaining: 2.37s
895:	learn: 1127.0693702	total: 20.3s	remaining: 2.35s
896:	learn: 1126.6009368	total: 20.3s	remaining: 2.33s
897:	learn: 1126.37630

In [26]:
pred_stacking

array([37841.32017553, 37915.07814542, 37751.7782129 , ...,
       36768.27887027, 37255.48037029, 38713.46747157])

In [ ]:
import numpy as np

predict = pd.DataFrame(pred_stacking).iloc[0:1416]

predict.to_csv("./submission.csv", index=True)
predict = pd.read_csv("./submission.csv")
predict.columns = ['id', 'count']
predict.to_csv("./submission.csv", index=False)